## 4(1) AutoML

### Import

In [1]:
import pandas as pd
from pycaret.regression import *

### Read Data

In [2]:
train = pd.read_parquet('../data/raw/train.parquet')
test = pd.read_parquet('../data/raw/test.parquet')

### SetUp

In [3]:
setup = setup(train, target='target', use_gpu=True,
              ignore_features=['id','vehicle_restricted','height_restricted'], 
              categorical_features = ['day_of_week','base_hour','road_in_use','road_rating','road_name','multi_linked',
                                      'connect_code','maximum_speed_limit','road_type',
                                      'start_turn_restricted','end_turn_restricted','start_node_name','end_node_name'],
              session_id=123, 
              transformation=True,
              combine_rare_levels=True, remove_multicollinearity=True, ignore_low_variance=True,feature_selection=True,
              fold=5, fold_strategy='stratifiedkfold', fold_shuffle=True)

IntProgress(value=0, description='Processing: ', max=3)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,09:21:44
Status,. . . . . . . . . . . . . . . . . .,Preprocessing Data


Text(value="Following data types have been inferred automatically, if they are correct press enter to continue…

,Data Type
base_date,Numeric
day_of_week,Categorical
base_hour,Categorical
road_in_use,Categorical
lane_count,Categorical
road_rating,Categorical
road_name,Categorical
multi_linked,Categorical
connect_code,Categorical
maximum_speed_limit,Categorical


MemoryError: Unable to allocate 17.3 GiB for an array with shape (985, 4701217) and data type float32

### Models

In [ ]:
clfs = compare_models(n_select=5, 
                      exclude=['svm', 'ridge', 'gbr'],  # 오래 걸리는 모델 제외 
                      sort='MAE', fold=5) 

In [ ]:
tuned_models = [tune_model(m, optimize='MAE', n_iter=30, fold=5) for m in clfs]

In [ ]:
blender = blend_models(estimator_list=tuned_models, method='auto', fold=5)

In [ ]:
best_model = automl(optimize='MAE')

In [ ]:
plot_model(best_model, plot='auc')

### Predict

In [ ]:
pred = predict_model(best_model, data=test) 
pred

In [ ]:
pd.DataFrame({'id': test.id, 'target': pred.Score}).to_csv('pycaret.csv', index=False)